 # Install Dependencies

 https://huggingface.co/tiiuae/falcon-40b

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
import locale
locale.getpreferredencoding = lambda x: "UTF-8"

In [3]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade

In [4]:
!pip install langchain einops accelerate transformers bitsandbytes

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


# Import Dependencies

In [5]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import os
import torch

/Users/fredericowieser/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Check if cuda is available
torch.cuda.is_available()

False

# Build the Pipeline

In [7]:
# Define Model ID
# model_id = "tiiuae/falcon-40b-instruct"

# model_id = "tiiuae/falcon-7b-instruct"
# model_id = "WizardLM/WizardCoder-Python-34B-V1.0"
model_id = "WizardLM/WizardCoder-Python-7B-V1.0"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Load Model
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir='./workspace/', trust_remote_code=True, device_map="auto", offload_folder="offload")

# Set PT model to inference mode
model.eval()

# Build HF Transformers pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=2000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards: 100%|██████████| 2/2 [01:04<00:00, 32.21s/it]


In [ ]:
print(results[0]["generated_text"])

who is Atatürk?
Atatürk was a Turkish revolutionary leader who fought for and successfully achieved independence for the Republic of Turkey from the Byzantine Empire. He later became the President of the Republic.


# Pass it to Langchain with Memory Module

In [ ]:
# Setup prompt template
template = PromptTemplate(input_variables=['input'], template='{input}')

# Pass hugging face pipeline to langchain class
llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, llm=llm)

# conversation.predict(input="Hi there!")

In [ ]:

results = conversation.predict(input="Where is Istanbul")

print(results)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 Istanbul is the capital and largest city of Turkey, located on the shores of the Bosporus Straits between the Aegean and Black Seas. It is also a transcontinental city, spanning the continents of Europe and Asia.

Current conversation:
[]
AI: Istanbul is located in Turkey, a transcontinental country spanning both the continents of Europe and Asia.


In [ ]:
print(results)

 Istanbul is the capital and largest city of Turkey, located on the shores of the Bosporus Straits between the Aegean and Black Seas. It is also a transcontinental city, spanning the continents of Europe and Asia.

Current conversation:
[]
AI: Istanbul is located in Turkey, a transcontinental country spanning both the continents of Europe and Asia.


In [ ]:

results = conversation.predict(input="Can I swim there?")

print(results)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 I do not know.

As an AI language model, I am unable to provide information on the current conditions of natural bodies of water. It is important to consult official and reliable sources for up-to-date safety information.


In [ ]:
results

' I do not know.\n\nAs an AI language model, I am unable to provide information on the current conditions of natural bodies of water. It is important to consult official and reliable sources for up-to-date safety information.'

In [ ]:
results = conversation.predict(input="is there hot in summer?")

print(results)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


# Build Gradio App

In [ ]:
# Install Gradio for the UI component
!pip install gradio

In [ ]:
# Import gradio for UI
import gradio as gr

In [ ]:
# Create generate function - this will be called when a user runs the gradio app
def generate(prompt):
    # The prompt will get passed to the LLM Chain!
    return conversation.predict(input=prompt)


In [ ]:
# Define a string variable to hold the title of the app
title = '🦜🔗 Falcon-7b-Instruct'
# Define another string variable to hold the description of the app
description = 'This application demonstrates the use of the open-source `Falcon-7b-Instruct` LLM with memory.'


In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = conversation.predict(input=message)
        chat_history.append((message, bot_message))
        print(chat_history)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)